<a href="https://colab.research.google.com/github/FG2511/ARE/blob/master/project/model1_cross_validation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
'''
@File name: model1_cross_validation.ipynb
@Created on 2018-12-20
@Authors: Federica Gerina, Francesca Moi, Silvia Maria Massa
@Description: Given a time-series dataset that contains minute-by-minute data 
about different kind of gases, collected by the uHoo air quality sensor, train
a NN that classifies if a minute belongs to the class "Pasto" (1) otherwise to
the class "Other" (0).
'''

!pip install liac-arff

import arff
import math

import numpy as np

from keras import optimizers
from keras.models import Sequential
from keras.models import load_model
from keras.layers import Dense, Dropout, LeakyReLU, BatchNormalization, Activation
from keras.callbacks import EarlyStopping

from sklearn.utils import compute_class_weight
from sklearn.model_selection import KFold
from sklearn.metrics import confusion_matrix

import sys
sys.path.append('local_modules')

#import postprocessing_sliding
#import postprocessing_Silvia
#import plotting

  Stored in directory: /root/.cache/pip/wheels/d1/6a/e7/529dc54d76ecede4346164a09ae3168df358945612710f5203
Successfully built liac-arff


Using TensorFlow backend.


In [0]:
#fix random seed for reproducibility
seed = 5
np.random.seed(seed)

In [0]:
'''
@Description: generate a multilayer perceptron with LeakyRelu as activation
function.
@param: 
  - shape : int, the shape of the input
  - n_features: int, the number of features given
'''

#MODELLO 1
#REGOLA: input/2, input, 2*input, 1
#layers TUTTE LE FEATURE: 57, 113, 226, 1 
#layers TIME CO2 TEMP: 21, 41, 82, 1 
#layers TIME CO2 TEMP PM25/TVOC: 30, 59, 118, 1 
#layers TIME CO2 TEMP PM25 TVOC: 39, 77, 154, 1 

def generate_model_leaky(shape, n_features):
  
  units_1 = int(n_features/2)
  units_2 = n_features
  units_3 = n_features*2
  
  model = Sequential()
  model.add(BatchNormalization())
  
  model.add(Dense(units_1, input_dim=shape, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(units_2, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(units_3, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(LeakyReLU(alpha = 0.2))
  model.add(Dropout(0.5))
  
  model.add(Dense(1, activation='sigmoid'))
  #print(model.summary())

  return model

In [0]:
'''
@Description: generate a multilayer perceptron with Relu as activation
function.
@param: 
  - shape : int, the shape of the input
  - n_features: int, the number of features given
'''

#MODELLO 2
#REGOLA: a= input, b= a*2/3+c, c= b*2/3+1
#layers TUTTE LE FEATURE: 113, 229, 153, 1
#layers TIME CO2 TEMP: 41, 85, 57, 1 
#layers TIME CO2 TEMP PM25/TVOC: 59, 121, 81, 1 
#layers TIME CO2 TEMP PM25 TVOC: 77, 157, 105, 1 

def generate_model(shape, n_features):
  
  a = np.array([[1,0,0],[-(2/3),1,-1],[0,-(2/3),1]])
  b = np.array([n_features,0,1])
  x = np.linalg.solve(a, b)
  
  units_1 = int(x[0])
  units_2 = int(x[1])
  units_3 = int(x[2])
  
  model = Sequential()
 
  model.add(Dense(units_1, input_dim=shape, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  model.add(Dense(units_2, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  model.add(Dense(units_3, kernel_initializer='random_uniform', use_bias = False))
  model.add(BatchNormalization())
  model.add(Activation('relu'))
  model.add(Dropout(0,5))
  
  model.add(Dense(1, activation='sigmoid'))
  #print(model.summary())

  return model

In [19]:
#@title SCEGLI I PARAMETRI

'''
@Description: MAIN
'''

#LOAD DATA
print("Loading data...")

dataset = '/root/data/uHooComplete_featureDataset(CO2_TEMP).arff' #@param {type:"string"}

with open (dataset, encoding='utf-8') as f:
  dataDictionary = arff.load(f)

data = np.array(dataDictionary['data'])
print("DATASET LOADED")

#CONVERTING VALUES
print("\nConverting values...")
for i in data:
  if(i[-1] == 'Other'): i[-1] = 0
  elif(i[-1] == 'Pasto') : i[-1] = 1

dataset = data.astype('float32')
print("CONVERSION DONE")

#SPLIT INTO INPUT (X) AND OUTPUT (Y) VARIABLES
s = dataset.shape[-1]
X = dataset[:,0:s-1]
Y = dataset[:,s-1]

n_features = s-1

#OPTIMIZERS
adm = optimizers.Adam(lr=0.0001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

#LOSS
loss = 'binary_crossentropy'

#MODEL
modello = 1 #@param {type:"integer"}

#DEFINE K-FOLD CROSS-VALIDATION
fold = 10 #@param {type:"integer"}

kfold = KFold(n_splits=fold, shuffle=False, random_state=None)

cvscores = []
predictions = []

test_time =[]

real = []
true_p =[]
true_n = []

dimSplit = math.floor(len(dataset[:,0])/fold)
startIndex = 0
finishIndex = dimSplit-1

i = 1

for train, test in kfold.split(X, Y):
  
  print("\nFOLD: %d" %i)
  
  #COMPUTE CLASS WEIGHT
  labels = np.unique(Y[train])
  classWeight = compute_class_weight('balanced', labels, Y[train])
  classWeight = dict(zip(labels,classWeight))

  #GENERATE MODEL
  print("\nGenerate model...")

  if modello==1 :
    model = generate_model_leaky(X[train].shape[-1], n_features)
  elif modello ==2:
    model = generate_model(X[train].shape[-1], n_features)

  #COMPILE MODEL
  print("\nCompile model...")
  model.compile(loss = loss, optimizer = adm , metrics=['accuracy'])

  #FIT MODEL
  print("\nFit model...")
  epoche = 6 #@param{type:"integer"}
  history = model.fit(X[train], Y[train], epochs=epoche, batch_size = 128, shuffle = True, verbose=1, class_weight = classWeight)

  #EVALUATE MODEL
  print("\nEvaluate model...")
  scores_test = model.evaluate(X[test], Y[test], batch_size= 128, verbose = 0)
  print("Test loss: %.2f%%" % (scores_test[0] * 100))
  print("Test accuracy: %.2f%%" % (scores_test[1] * 100))
  
  cvscores.append(scores_test[1] * 100)
  
  #CALCULATE PREDICTIONS
  print("\nCalculate predictions...")
  pred = model.predict_classes(X[test], batch_size=128, verbose=0)
  flat_pred = [item for sublist in pred for item in sublist]
  predictions.append(flat_pred)
  
  real.append(Y[test])
  
  #STORE DATETIME
  time = []  
  for j in X[test]:
    time.append(int(j[-5]))  
  test_time.append(time)
  
  #PLOTTING
  #plotting.plot_model_results(history)
  #plotting.plot_co2_temp_cross(flat_pred, dimSplit, startIndex, finishIndex, i)
  
  #CONFUSION MATRIX
  print("\nCompute confusion matrix...")
  y_true = Y[test]
  tn, fp, fn, tp = confusion_matrix(y_true, pred).ravel()
  (tn, fp, fn, tp)
  other = 100*tn/(tn+fp)
  pasto = 100*tp/(fn+tp)
  true_p.append(pasto)
  true_n.append(other)
  print("Other: %.2f %%" % other)
  print("Pasto: %.2f %%" % pasto)
  
  i+=1

  
print("MEAN ACCURACY: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(cvscores), np.std(cvscores)))
print("MEAN TRUE POSITIVE RATE: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(true_p), np.std(true_p)))
print("MEAN TRUE NEGATIVE RATE: %.2f%% (STANDARD DEVIATION: +/- %.2f%%)" % (np.mean(true_n), np.std(true_n)))


Loading data...
DATASET LOADED

Converting values...
CONVERSION DONE

FOLD: 1

Generate model...

Compile model...

Fit model...
Epoch 1/6
315441/315441 [==============================] - 11s 34us/step - loss: 0.5510 - acc: 0.7523
Epoch 2/6
315441/315441 [==============================] - 9s 28us/step - loss: 0.4549 - acc: 0.8387
Epoch 3/6
315441/315441 [==============================] - 9s 27us/step - loss: 0.4345 - acc: 0.8407
Epoch 4/6
315441/315441 [==============================] - 9s 27us/step - loss: 0.4264 - acc: 0.8404
Epoch 5/6
315441/315441 [==============================] - 9s 27us/step - loss: 0.4223 - acc: 0.8404
Epoch 6/6
315441/315441 [==============================] - 9s 27us/step - loss: 0.4157 - acc: 0.8407

Evaluate model...
Test loss: 34.13%
Test accuracy: 92.26%

Calculate predictions...

Compute confusion matrix...
Other: 94.31 %
Pasto: 42.68 %

FOLD: 2

Generate model...

Compile model...

Fit model...
Epoch 1/6
315442/315442 [==============================] - 1

In [20]:
test_time

[[30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62,
  63,
  64,
  65,
  66,
  67,
  68,
  68,
  69,
  70,
  71,
  72,
  73,
  74,
  75,
  76,
  77,
  78,
  79,
  80,
  81,
  82,
  83,
  84,
  85,
  86,
  87,
  88,
  89,
  90,
  91,
  92,
  93,
  94,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
  105,
  106,
  107,
  108,
  109,
  110,
  111,
  112,
  113,
  114,
  115,
  116,
  117,
  118,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  126,
  127,
  127,
  128,
  129,
  130,
  131,
  132,
  133,
  134,
  135,
  136,
  137,
  138,
  139,
  140,
  141,
  142,
  143,
  144,
  145,
  146,
  147,
  148,
  149,
  150,
  151,
  152,
  153,
  154,
  155,
  156,
  157,
  158,
  159,
  160,
  161,
  162,
  163,
  164,
  165,
  166,
  167,
  168,
  169,
  170,
  171,
  172,
  173,
  174,
  175,
  176,
  177,
  178,
  179,
  180,


In [0]:
#@title SCEGLI maxWidth
'''
@Description: POST PROCESSING
'''
maxWidth = 6 #@param {type:"integer"}

flat_pred = predictions
flat_pred = [item for sublist in flat_pred for item in sublist]

#CONFUSION MATRIX
print("\n Confusion matrix pre postprocessing...")
y_true = [item for sublist in real for item in sublist]
y_true = np.array(y_true,dtype=np.int)
n_y_pred = flat_pred 
tn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)

#new_pred = postprocessing_sliding.sliding_windows(flat_pred)
new_pred = postprocessing_Silvia.setWidth(maxWidth,flat_pred)

#CONFUSION MATRIX
print("\n Confusion matrix post postprocessing...")
y_true = [item for sublist in real for item in sublist]
y_true = np.array(y_true,dtype=np.int)
n_y_pred = new_pred 
tn, fp, fn, tp = confusion_matrix(y_true, n_y_pred).ravel()
print("TN", tn)
print("FP", fp)
print("FN", fn)
print("TP", tp)
other = 100*tn/(tn+fp)
pasto = 100*tp/(fn+tp)
print("Other corretti: %.2f %%" % other)
print("Pasto corretti: %.2f %%" % pasto)